## Analyse des performances des modèles DIST_REL_CC_01, DIST_REL_OH_01 et DIST_REL_CH_01

Nous allons dans ce notebook analyser les performances des modèles DIST_REL_CC_01, DIST_REL_OH_01 et DIST_REL_CH_01, entraînés dans le notebook 12.1.

Pour chaque modèle, nous allons calculer et afficher trois représentations graphiques :

* La distribution des erreurs, pour représenter la capacité du modèle à prédire des valeurs correctes en général.
* La distribution des erreurs en fonction des valeurs cibles, pour déterminer les plages de valeurs cibles que les modèles prédisent plus ou moins bien.
* Les prédictions en fonction des valeurs cibles, pour comparer les prédictions aux valeurs cibles sur toutes les plages.

De plus, nous allons calculer pour chaque modèle certaines statistiques sur la distribution des erreurs.

#### Chemin des fichiers

In [2]:
CC_model_loc = "../models/DIST_REL_CC_01/12.1/DIST_REL_CC_01_basic.tflearn"
CH_model_loc = "../models/DIST_REL_CH_01/12.1/DIST_REL_CH_01_basic.tflearn"
OH_model_loc = "../models/DIST_REL_OH_01/12.1/DIST_REL_OH_01_basic.tflearn"

CC_test_set_prepared_input_loc = "../data/DIST_REL_CC/test_set_prepared_input.h5"
CC_test_set_labels_loc = "../data/DIST_REL_CC/test_set_labels.h5"

CH_test_set_prepared_input_loc = "../data/DIST_REL_CH/test_set_prepared_input.h5"
CH_test_set_labels_loc = "../data/DIST_REL_CH/test_set_labels.h5"

OH_test_set_prepared_input_loc = "../data/DIST_REL_OH/test_set_prepared_input.h5"
OH_test_set_labels_loc = "../data/DIST_REL_OH/test_set_labels.h5"

#### Constantes

In [1]:
C_ANUM = 6.
O_ANUM = 8.
H_ANUM = 1.

### Création du RN

#### Fonction de coût (RMSE)

In [3]:
def rmse(pred, targets):
    with tf.name_scope("rmse_loss"):
        return tf.sqrt(tf.reduce_mean(tf.squared_difference(pred, targets)), name="rmse")

#### Fonction d'évaluation des performances (opposé du RMSE)

In [4]:
def rmse_valid(pred, targets, inputs):
    with tf.name_scope("rmse_validation"):
        return -rmse(pred, targets)

#### Fonction de création du RN


In [5]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.optimizers import Adam
from tflearn.data_preprocessing import DataPreprocessing
import math


def creer_RN(epsilon=1e-8, learning_rate=0.001, dropout_val=0.99, stddev_init=0.001,
             hidden_act='relu', outlayer_act='prelu', weight_decay=0.001, width=870, depth=3,
             validation_fun=rmse_valid, cost_fun=rmse):

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)
    
    # On créé l'input du RN
    network = input_data(shape=[None, 870], name='input')
    
    # On créé les couches cachées
    for i in range(depth):
        network = fully_connected(network, width, activation=hidden_act, name='fc'+str(i), weights_init=winit,
                                  weight_decay=weight_decay)
        # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
        network = dropout(network, dropout_val)
    
    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 1, activation=outlayer_act, name='outlayer', weights_init=winit)
    
    adam = Adam(learning_rate=learning_rate, epsilon=epsilon)
    
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer=adam,
    loss=cost_fun, metric=validation_fun, name='target')
            
    return network

/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


### Calcul des prédictions

#### Calcul du RMSE sur des vecteurs

In [11]:
import numpy as np

def rmse_test(targets, predictions):
    return np.sqrt(np.square(np.diff([targets, predictions], axis=0)))[0]

#### Calcul de l'erreur sur chaque exemple du jeu de test

In [12]:
import h5py
from tflearn.data_utils import pad_sequences
import tensorflow as tf
import tflearn as tfl

def calculer_pred(model_loc, test_set_prepared_input_loc, test_set_labels_loc):
    """ Renvoie l'erreur et la prédiction pour chaque exemple du jeu de test """
    
    tf.reset_default_graph()
    
    # Importation du modèle
    network = creer_RN(learning_rate=0.01, dropout_val=0.98, epsilon=0.001, hidden_act="elu", outlayer_act="linear", width=870, depth=3)
    model = tfl.DNN(network)
    model.load(model_loc, weights_only=True)
    
    # Chargement des données
    inputs_h5 = h5py.File(test_set_prepared_input_loc, 'r')
    targets_h5 = h5py.File(test_set_labels_loc, 'r')
    
    inputs = np.array(inputs_h5["inputs"])
    inputs = pad_sequences(inputs, dtype="float32", maxlen=870)
    
    targets = np.array(targets_h5["targets"])
    targets = targets.reshape(-1, 1)
        
    predictions = []
    
    i = 0
    batch_size = 10000
    
    while i < len(inputs):
        j = min(len(inputs), i+batch_size)
        predictions.extend(model.predict(np.array(inputs[i:j]).reshape(-1, 870)))
        i += batch_size
    
    
    predictions = np.array(predictions).reshape(-1, 1)
        
    return rmse_test(targets, predictions).reshape(1, -1)[0], predictions
    
    

### Définitions des fonctions de statistiques et de plot

#### Affichage statistiques

In [7]:
def print_stats_rmse(rmses):
    print("Erreur moyenne : "+str(np.mean(rmses)))
    print("Erreur médiane : "+str(np.median(rmses)))
    print("Écart-type : "+str(np.std(rmses)))
    print("Erreur min : "+str(min(rmses)))
    print("Erreur max : "+str(max(rmses)))
    

#### Représentation graphique de la distribution des erreurs selon leur valeur

In [8]:
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['agg.path.chunksize'] = 10000


def plot_distrib_rmses_val(rmses, padding, model_name, figures_loc):
    """ Affiche la distribution des rmses selon un pas donné """
    
    # On calcule les bornes de notre index
    debut_index = np.floor(min(rmses))
    fin_index = max(rmses)+padding
    
    # On créé l'index des rmses
    index = np.arange(debut_index, fin_index, padding)
    
    # On créé le compteur de rmses
    rmse_occ = np.zeros_like(index)
        
    # Calcul des rmse
    for rmse in rmses:
        rmse_occ[int((rmse-debut_index)/padding)] += 1
    
    fig = plt.figure()
    
    
    ## Échelle linéaire ##
    ax1 = fig.add_subplot(121)
    ax1.set_title("Modèle "+model_name+"\n Distribution des erreurs")
    ax1.set_xlabel("Erreur absolue (mÅ)")
    ax1.set_ylabel("Occurrences sur le jeu de test")
    ax1.bar(index, rmse_occ)
   

    ## Échelle logarithmique
    ax2 = fig.add_subplot(122)
    ax2.set_title("Modèle "+model_name+"\n Distribution des erreurs")
    ax2.set_xlabel("Erreur absolue (mÅ)")
    ax2.set_ylabel("Occurrences sur le jeu de test (échelle logarithmique)")
    ax2.set_yscale("log")
    ax2.bar(index, rmse_occ)
    
    plt.gcf().subplots_adjust(wspace=0.3)

    
    plt.savefig(figures_loc+model_name+"_distrib_rmse_val.png", dpi=250)
    plt.show()


#### Distribution des erreurs selon la distance à prédire

In [9]:
def plot_rmse_distrib_dist(rmses, targets, anum_1, anum_2, model_name, figures_loc):
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    ax.set_title("Modèle "+model_name+"\nDistribution des erreurs selon les distances cibles")
    ax.set_xlabel("Distance cible (mÅ)")
    ax.set_ylabel("Erreur absolue (mÅ)")
    ax.plot(targets, rmses, ",")
    
    ax.annotate('', xy=(0.07, 0.89), xytext=(0.17, 0.89), xycoords=ax.transAxes,
            arrowprops=dict(facecolor='red', shrink=0.05)
            )
    
    ax.text(0.19, 0.85, "Longueur de liaison\ntypique", transform=ax.transAxes)
    
    
    if anum_1==C_ANUM and anum_2==C_ANUM:
        
        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1200,0)))[0]
        ax.annotate('triple', xy=(x_axe_coord, 0), xytext=(x_axe_coord-0.1, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05), 
                )

        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1340,0)))[0]
        ax.annotate('double', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05),
                )

        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1540,0)))[0]
        ax.annotate('simple', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05),
                )

        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1400,0)))[0]
        ax.annotate('aromatique', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05),
                )

    elif anum_1==C_ANUM and anum_2==H_ANUM:
        
        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1090,0)))[0]
        ax.annotate('simple', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05), 
                )
        
    elif anum_1==O_ANUM and anum_2==H_ANUM:
        
        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((980,0)))[0]
        ax.annotate('simple', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05), 
                )
        
        
    plt.savefig(figures_loc+model_name+"_distrib_rmse_dist.png", dpi=250)
    
    plt.show()
    


#### Visualisation des prédictions selon les distances attendues

In [10]:
def fun_id(x):
    return x

def plot_targets_pred(targets, preds, anum_1, anum_2, model_name, figures_loc):
        
    fig = plt.figure()
    ax = fig.add_subplot(111)
        
    ax.set_title("Modèle "+model_name+"\n Prédictions en fonction des distances cibles")
    ax.set_xlabel("Distance cible (mÅ)")
    ax.set_ylabel("Distance prédite (mÅ)")
    
    ax.plot(targets, preds, ",")
    
    """
    ax.annotate('', xy=(0.07, 0.89), xytext=(0.17, 0.89), xycoords=ax.transAxes,
            arrowprops=dict(facecolor='red', shrink=0.05)
            )
    
    ax.text(0.19, 0.85, "Longueur de liaison\ntypique", transform=ax.transAxes)
    """
    if anum_1==C_ANUM and anum_2==C_ANUM:
        
        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1200,0)))[0]
        ax.annotate('triple', xy=(x_axe_coord, 0), xytext=(x_axe_coord+0.06, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05), label="Longeur de liaison typique"
                )

        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1340,0)))[0]
        ax.annotate('double', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05),
                )

        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1540,0)))[0]
        ax.annotate('simple', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05),
                )

        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1400,0)))[0]
        ax.annotate('aromatique', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05),
                )

    elif anum_1==C_ANUM and anum_2==H_ANUM:
        
        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((1090,0)))[0]
        ax.annotate('simple', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05), label="Longueur de liaison typique"
                )
        
    elif anum_1==O_ANUM and anum_2==H_ANUM:
        
        x_axe_coord = ax.transAxes.inverted().transform(ax.transData.transform((980,0)))[0]
        ax.annotate('simple', xy=(x_axe_coord, 0), xytext=(x_axe_coord, 0.1), xycoords=ax.transAxes,
                arrowprops=dict(facecolor='red', shrink=0.05), label="Longueur de liaison typique"
                )
        
        
    # On trace la droite y=x
    x=np.linspace(min(targets), max(targets))
    ax.plot(x, fun_id(x), color='darkgreen', label="Modèle théorique parfait")
    
    ax.text(0.65, 0.85, 'Modèle théorique\nparfait',
        transform=ax.transAxes,
        color='darkgreen')
    
    ax.legend(loc='upper center', shadow=True)
    
    plt.savefig(figures_loc+model_name+"_preds_fonction_cibles.png", dpi=250)
    
    plt.show()



## Analyse des modèles

### Modèle DIST_REL_CC_01

#### Calcul des prédictions

In [ ]:
rmses, predictions = calculer_pred(CC_model_loc, CC_test_set_prepared_input_loc, CC_test_set_labels_loc)

#### Récupération du vecteur cibles (distances)

In [ ]:
def extract_targets(test_set_labels_loc):
    targets_h5 = h5py.File(test_set_labels_loc, 'r')
    targets = np.array(targets_h5["targets"])
    targets = targets.reshape(-1, 1)
    return targets

#targets = extract_targets(CC_test_set_labels_loc)

#### Affichage des statistiques sur les erreurs

In [ ]:
#print_stats_rmse(rmses)

##### Sortie
```
Plotting DIST_REL_CC_01
Dataset size : 1106823
Mean error : 0.83337665
Median error : 0.46037596
Standard deviation : 1.20663
Min error : 0.0
Max error : 30.113806
Relative error : 0.5709485%
```



#### Distribution des erreurs

In [ ]:
#plot_distrib_rmses_val(rmses, 1, "DIST_REL_CC_01", "../figures/12.2-")

![title](../figures/12.2-DIST_REL_CC_01_distrib_rmse_val.png)

#### Distribution des erreurs selon les cibles

In [ ]:
#plot_rmse_distrib_dist(rmses, targets, C_ANUM, C_ANUM, "DIST_REL_CC_01", "../figures/12.2-")

![title](../figures/12.2-DIST_REL_CC_01_distrib_rmse_dist.png)

#### Prédictions selon les distances cibles

In [ ]:
#plot_targets_pred(targets, predictions, C_ANUM, C_ANUM, "DIST_REL_CC_01", "../figures/12.2-")

![title](../figures/12.2-DIST_REL_CC_01_preds_fonction_cibles.png)

## Modèle DIST_REL_OH_01

#### Calcul des prédictions

In [ ]:
#rmses, predictions = calculer_pred(OH_model_loc, OH_test_set_prepared_input_loc, OH_test_set_labels_loc)

#### Récupération du vecteur cibles (distances)

In [ ]:
#targets = extract_targets(OH_test_set_labels_loc)

#### Affichage des statistiques sur les erreurs

In [ ]:
#print_stats_rmse(rmses)

##### Sortie

```
Plotting DIST_REL_OH_01
Dataset size : 143588
Mean error : 0.19465849
Median error : 0.11533813
Standard deviation : 0.251944
Min error : 0.0
Max error : 7.2529726
Relative error : 0.19857097%
```

#### Distribution des erreurs

In [ ]:
#plot_distrib_rmses_val(rmses, 1, "DIST_REL_OH_01", "../figures/12.2-")

![title](../figures/12.2-DIST_REL_OH_01_distrib_rmse_val.png)

#### Distribution des erreurs selon les cibles

In [ ]:
#plot_rmse_distrib_dist(rmses, targets, O_ANUM, H_ANUM, "DIST_REL_OH_01", "../figures/12.2-")

![title](../figures/12.2-DIST_REL_OH_01_distrib_rmse_dist.png)

#### Prédictions selon les distances cibles

In [ ]:
#plot_targets_pred(targets, predictions, O_ANUM, H_ANUM, "DIST_REL_OH_01", "../figures/12.2-")

![title](../figures/12.2-DIST_REL_OH_01_preds_fonction_cibles.png)

## Modèle DIST_REL_CH_01

#### Calcul des prédictions

In [ ]:
#rmses, predictions = calculer_pred(CH_model_loc, CH_test_set_prepared_input_loc, CH_test_set_labels_loc)

#### Récupération du vecteur cibles (distances)

In [ ]:
#targets = extract_targets(CH_test_set_labels_loc)

#### Affichage des statistiques sur les erreurs

In [ ]:
#print_stats_rmse(rmses)

##### Sortie
```
Plotting DIST_REL_CH_01
Dataset size : 1158251
Mean error : 0.17527544
Median error : 0.11315918
    Standard deviation : 0.19614318
Min error : 0.0
Max error : 22.14729
Relative error : 0.15991603%
```

#### Distribution des erreurs

In [ ]:
#plot_distrib_rmses_val(rmses, 1, "DIST_REL_CH_01", "../figures/12.2-")

![title](../figures/12.2-DIST_REL_CH_01_distrib_rmse_val.png)

#### Distribution des erreurs selon les cibles

In [ ]:
#plot_rmse_distrib_dist(rmses, targets, C_ANUM, H_ANUM, "DIST_REL_CH_01", "../figures/12.2-")

![title](../figures/12.2-DIST_REL_CH_01_distrib_rmse_dist.png)

#### Prédictions selon les distances cibles

In [ ]:
"""plot_targets_pred(targets, predictions, C_ANUM, H_ANUM, "DIST_REL_CH_01", "../figures/12.2-")"""

![title](../figures/12.2-DIST_REL_CH_01_preds_fonction_cibles.png)

## Recherche des molécules donnant les moins bons résultats

Afin d'essayer de comprendre pourquoi les prédictions des modèles sont moins bonnes sur certaines molécules, nous allons rechercher les identifiants pubchem des exemples des jeux de test menant aux moins bons résultats pour chaque modèle.

In [6]:
import h5py
import numpy as np

def affichage_mol_mauvais_results(model_loc, prepared_input_loc, labels_loc, X_anum, Y_anum, X_name, Y_name):
    
    
    # Calcul des prédictions
    rmses, predictions = calculer_pred(model_loc, prepared_input_loc, labels_loc)

    # Chargement des données concernant les identifiants des molécules
    labels_h5 = h5py.File(labels_loc, 'r')
    prepared_input_h5 = h5py.File(prepared_input_loc, 'r')
    
    targets_np = np.array(labels_h5["targets"])
    ids_np = np.array(prepared_input_h5["pubchem_ids"])
    prepared_input_np = np.array(prepared_input_h5["inputs"])
    
    
    if X_anum==C_ANUM and Y_anum==C_ANUM:
        seuil = 250
    elif X_anum==O_ANUM and Y_anum==H_ANUM:
        seuil = 55
    elif X_anum==C_ANUM and Y_anum==H_ANUM:
        seuil = 75
        
    print("Affichage des ID pubchem associés aux moins bons résultats pour les liaisons "+X_name+Y_name+
          "err>"+str(seuil))
    
    for i, err in enumerate(rmses):
        
        if err>seuil:
            print("Erreur : "+str(err)+" mÅ")
            print("ID pubchem :"+str(ids_np[i]))
            print("Distance cible : "+str(targets_np[i])+" mÅ")
            print()
    

### Recherche sur le modèle DIST_REL_CC_01

In [ ]:
affichage_mol_mauvais_results(CC_model_loc, CC_test_set_prepared_input_loc, CC_test_set_labels_loc, C_ANUM, 
                             C_ANUM, "C", "C")

### Recherche sur le modèle DIST_REL_OH_01

In [ ]:
affichage_mol_mauvais_results(OH_model_loc, OH_test_set_prepared_input_loc, OH_test_set_labels_loc, O_ANUM, 
                             H_ANUM, "O", "H")

### Recherche sur le modèle DIST_REL_CH_01

In [ ]:
affichage_mol_mauvais_results(CH_model_loc, CH_test_set_prepared_input_loc, CH_test_set_labels_loc, C_ANUM, 
                             H_ANUM, "C", "H")